In [235]:
import numpy as np
import math
from scipy.linalg import lstsq
from sklearn.metrics import mean_squared_error

In [257]:
# Signal and channel parameters (as per your input)
Fs = 100000    # Sampling frequency
Fc = 2e9          # Carrier frequency
Fd = 100*(100 / Fs)   # Doppler frequency = 100
Tc = 1 / (10 * Fd)  # Coherence time based on Doppler frequency
N = 100000        # Number of data points in the signal
M = 3             # Number of different channel values (multi-path)
packet_size = math.floor(Fs * Tc)//5  # Size of each packet
header_packet_size = math.floor((1 / 6) * packet_size)  # Known part of the signal (header)
noise_SNR = 10
noise_level = 1 / noise_SNR  # Noise level (1/SNR)

In [259]:
# Generate BPSK signal
def generate_bpsk_signal(N):
    bits = np.random.randint(0, 2, N)
    return 2 * bits - 1  # BPSK mapping: 0 -> -1, 1 -> 1


In [261]:
# Sum-of-sines model for Rayleigh fading channel
def sum_of_sines_model(N, Fd, Fs):
    t = np.arange(N) / Fs
    num_sines = 8  # Number of sinusoids
    doppler_shift = np.linspace(-Fd, Fd, num_sines)
    phases = np.random.uniform(0, 2 * np.pi, num_sines)
    channel = np.zeros(N, dtype=complex)
    
    for i in range(num_sines):
        channel += np.exp(1j * (2 * np.pi * doppler_shift[i] * t + phases[i]))
    
    channel /= np.sqrt(num_sines)  # Normalize power
    return channel

In [263]:
# Add noise to the signal based on SNR
def add_noise(signal, snr_db):
    snr_linear = 10**(snr_db / 10)
    signal_power = np.mean(np.abs(signal)**2)
    noise_power = signal_power / snr_linear
    noise = np.sqrt(noise_power / 2) * (
        np.random.randn(*signal.shape) + 1j * np.random.randn(*signal.shape)
    )
    return signal + noise

In [265]:
# Compute BER
def compute_ber(original_bits, estimated_bits):
    errors = np.sum(original_bits != estimated_bits)
    return errors / len(original_bits)


In [267]:
# BPSK demodulation
def bpsk_demodulate(signal):
    return np.where(np.real(signal) >= 0, 1, -1)

In [269]:
# Main function to calculate BER and MSE for M independent channels
def calculate_ber_mse(M, N, Fd, Fs, packet_size, header_packet_size, noise_level):
    signal = generate_bpsk_signal(N)  # Generate BPSK signal
    num_packets = N // packet_size     # Number of packets
    
    for i in range(M):
        print(f"\nChannel {i + 1}:")
        
        # Generate Rayleigh fading channel
        channel = sum_of_sines_model(packet_size, Fd, Fs)
        
        total_ber = 0
        total_mse = 0
        total_bits = 0
        total_header_bits = 0
        
        # Process each packet
        for packet_idx in range(num_packets):
            # Get the start and end of the current packet
            packet_start = packet_idx * packet_size
            packet_end = packet_start + packet_size
            
            # Current packet of signal and assume constant channel for the packet
            packet_signal = signal[packet_start:packet_end]
            packet_channel = np.full(packet_size, channel[packet_idx])
            
            # Transmit signal through channel
            transmitted_signal = packet_signal * packet_channel
            
            # Add noise
            noisy_signal = add_noise(transmitted_signal, noise_level)
            
            # Least squares estimation over the header part
            header_start = 0
            header_end = header_packet_size
            header_transmitted = packet_signal[header_start:header_end]
            header_noisy = noisy_signal[header_start:header_end]
            
            # Perform least squares estimation to estimate the channel
            channel_estimated, _, _, _ = lstsq(header_transmitted.reshape(-1, 1), header_noisy)
            
            # Equalization for the entire packet
            equalized_signal = noisy_signal * np.conj(channel_estimated) / np.abs(channel_estimated)**2
            
            # BPSK demodulation
            demodulated_signal = bpsk_demodulate(equalized_signal)
            
            # Compute BER for the packet (excluding header)
            data_start = header_packet_size
            data_end = packet_size
            ber = compute_ber(packet_signal[data_start:data_end], demodulated_signal[data_start:data_end])
            
            # Compute MSE for the header
            mse = mean_squared_error(np.abs(packet_channel[header_start:header_end]), np.abs(np.full(header_packet_size, channel_estimated)))
            
            # Update total counts
            total_ber += ber * (data_end - data_start)
            total_mse += mse * header_packet_size
            total_bits += (data_end - data_start)
            total_header_bits += header_packet_size
        
        # Final BER and MSE for this channel
        final_ber = total_ber / total_bits
        final_mse = total_mse / total_header_bits
        
        # Print results
        print(f"  BER: {final_ber:.9f}")
        print(f"  MSE: {final_mse:.9f}")


In [271]:
# Call the function to calculate and print BER and MSE for M channels
calculate_ber_mse(M, N, Fd, Fs, packet_size, header_packet_size, noise_level)


Channel 1:
  BER: 0.077266455
  MSE: 0.000201497

Channel 2:
  BER: 0.075394492
  MSE: 0.000037073

Channel 3:
  BER: 0.075238495
  MSE: 0.000038802
